<a href="https://colab.research.google.com/github/eyang9001/NU_Radiology/blob/master/NU_Rad_Model_3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!pip install --upgrade wandb
!wandb login 3aa7250d739679a5cc8a09410800a87bc578ba26

     |████████████████████████████████| 1.4MB 2.7MB/s 
     |████████████████████████████████| 460kB 16.6MB/s 
     |████████████████████████████████| 102kB 9.1MB/s 
     |████████████████████████████████| 112kB 18.7MB/s 
     |████████████████████████████████| 102kB 9.1MB/s 
     |████████████████████████████████| 71kB 8.0MB/s 
     |████████████████████████████████| 71kB 7.0MB/s 
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=8fa13c9cb806b85f2e6bef9aa29dee6c1f30c5507f3910436e3353f68e76c2f6
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
  Created wheel for watchdog: filename=watchdog-0.10.2-cp36-none-any.whl size=73605 sha256=dd29214106030b180ca43ff5dd88a69bda9fc3d7e7af8a36bcc50065e19e77b6
  Stored in directory: /root/.cache/pip/wheels/bc/ed/6c/028dea90d31b359cd2a7c8b0da4db80e41d24a59614154072e
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=e89d

In [3]:
from sys import platform as sys_pf
# if sys_pf == 'darwin':
#     import matplotlib
#     matplotlib.use("TkAgg")
! pip install simpleitk
%matplotlib inline  
import os
import numpy as np
from matplotlib import pyplot as plt
import matplotlib
import SimpleITK as sitk
from scipy import ndimage
import random
import math
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Concatenate, Add, Average, Input, Dense, Flatten, BatchNormalization, Activation, LeakyReLU, Reshape
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D
from tensorflow.keras import backend as K
import tensorflow.keras.callbacks as callbacks
import tensorflow.keras.optimizers as optimizers
from sklearn import preprocessing as prepro
from sklearn.model_selection import train_test_split

from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

     |████████████████████████████████| 42.5MB 79kB/s 


Using TensorFlow backend.


In [0]:
image_shape = (256,256,176)

# Read in transformed images
def read_file(filename):
    data = np.loadtxt(filename, delimiter=",")
    data = np.reshape(data,image_shape)
    return data

# Read in non-transformed images
def read_original(filename):
    reader = sitk.ImageFileReader()
    reader.SetImageIO("NiftiImageIO")
    reader.SetFileName(filename)
    image = reader.Execute()

    # img1 = sitk.ReadImage(folder + item)  # alternative way to pull in image

    # convert image into np array & perform fft
    img = sitk.GetArrayFromImage(image)
    # Transpose the image so the first axis is Anterior-Posterior
    img = np.transpose(img, (2,1,0))
    return img 

def visualize(orig, new):
    plt.figure(figsize= (20,20))
    plt.subplot(121), plt.imshow(orig, cmap='gray')
    plt.title('Original'), plt.xticks([]), plt.yticks([])
    plt.subplot(122), plt.imshow(new, cmap='gray')
    plt.title('New'), plt.xticks([]), plt.yticks([])
    
def visualize3(orig, blur, predict):
    plt.figure(figsize= (30,30))
    plt.subplot(131), plt.imshow(orig, cmap='gray')
    plt.title('Original'), plt.xticks([]), plt.yticks([])
    plt.subplot(132), plt.imshow(blur, cmap='gray')
    plt.title('Blurred'), plt.xticks([]), plt.yticks([])
    plt.subplot(133), plt.imshow(predict, cmap='gray')
    plt.title('Model Output'), plt.xticks([]), plt.yticks([])

def visualize_model(model, ind, X, y, save=False, fn=''):
    x_input = X[ind]
    out = model.predict(np.array(x_input[np.newaxis, ...]))
    visualize3(y[ind][...,0], x_input[...,0], out[0,...,0])
    if save:
        plt.savefig(fn)
def generate_data(t_filenames, o_filenames, min_slc = 70, max_slc = 220):
    # Takes in the files of the transformed images and the originals to create the pairs
    # Also sets the min and max slice numbers to take in
    x_data = []
    y_data = []
    for i in range(len(t_filenames)):
        print('Generating data for: ' + t_filenames[i])
        x_mri = read_file(t_filenames[i])
        y_mri = read_original(o_filenames[i])
        for ii in np.arange(min_slc, max_slc+1):
            x_data.append((x_mri[ii] - np.amin(x_mri[ii]))/(np.amax(x_mri[ii]) - np.amin(x_mri[ii]))) # normalizes the intensity to between 0 and 1
            y_data.append((y_mri[ii] - np.amin(y_mri[ii]))/(np.amax(y_mri[ii]) - np.amin(y_mri[ii])))
    # Shape of x & y will be [# slices, 256, 176, 1]
    x_data = np.array(x_data)[..., np.newaxis]
    y_data = np.array(y_data)[..., np.newaxis]
    return x_data, y_data
def generate_data_3d(t_filenames, o_filenames):
    # Takes in the files of the transformed images and the originals to create the pairs
    # Also sets the min and max slice numbers to take in
    x_data = []
    y_data = []
    for i in range(len(t_filenames)):
        print('Generating data for: ' + t_filenames[i])
        x_mri = read_file(t_filenames[i])
        y_mri = read_original(o_filenames[i])
        x_data.append((x_mri - np.amin(x_mri))/(np.amax(x_mri) - np.amin(x_mri))) # normalizes the intensity to between 0 and 1
        y_data.append((y_mri - np.amin(y_mri))/(np.amax(y_mri) - np.amin(y_mri)))
    # Shape of x & y will be [# mris, 256, 256, 176, 1]
    x_data = np.array(x_data)[..., np.newaxis]
    y_data = np.array(y_data)[..., np.newaxis]
    return x_data, y_data

In [5]:
# generate 3d training data
trans_folder = 'gdrive/My Drive/NU_Rad/transforms/Training/'
orig_folder = 'gdrive/My Drive/NU_Rad/mris/'
X_filenames = []
y_filenames = []
for item in os.listdir(trans_folder):
    if item.endswith("_trans.txt.gz"):
        X_filenames.append(trans_folder + item)
        y_filenames.append(orig_folder + item[:3] + '.nii')

# X, y = generate_data(['gdrive/My Drive/NU_Rad/transforms/M02_motion5_trans.txt.gz'], ['gdrive/My Drive/NU_Rad/mris/M02.nii'])
X_train, y_train = generate_data_3d(X_filenames, y_filenames)

# generate 3d validation data
trans_folder = 'gdrive/My Drive/NU_Rad/transforms/Validation/'
X_filenames = []
y_filenames = []
for item in os.listdir(trans_folder):
    if item.endswith("_trans.txt.gz"):
        X_filenames.append(trans_folder + item)
        y_filenames.append(orig_folder + item[:3] + '.nii')
X_val, y_val = generate_data_3d(X_filenames, y_filenames)

Generating data for: gdrive/My Drive/NU_Rad/transforms/Training/M02_motion5_trans.txt.gz
Generating data for: gdrive/My Drive/NU_Rad/transforms/Training/M02_motion3_trans.txt.gz
Generating data for: gdrive/My Drive/NU_Rad/transforms/Training/M02_motion4_trans.txt.gz
Generating data for: gdrive/My Drive/NU_Rad/transforms/Training/M02_motion1_trans.txt.gz
Generating data for: gdrive/My Drive/NU_Rad/transforms/Training/M02_motion2_trans.txt.gz
Generating data for: gdrive/My Drive/NU_Rad/transforms/Training/M01_motion5_trans.txt.gz
Generating data for: gdrive/My Drive/NU_Rad/transforms/Training/M01_motion3_trans.txt.gz
Generating data for: gdrive/My Drive/NU_Rad/transforms/Training/M01_motion4_trans.txt.gz
Generating data for: gdrive/My Drive/NU_Rad/transforms/Training/M03_motion5_trans.txt.gz
Generating data for: gdrive/My Drive/NU_Rad/transforms/Training/M03_motion3_trans.txt.gz
Generating data for: gdrive/My Drive/NU_Rad/transforms/Training/M03_motion4_trans.txt.gz
Generating data for: 

In [0]:
# Simple 3d unet with mse loss

def conv_layer(filts, dim):
    # abstracted a single conv layer out since the parameters outside of dimension were kept the same
    return Conv3D(filts, dim, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')

# u_net model
def unet_model(lr= 1e-4, input_size = (256, 256, 176, 1), dropout_level = 0.1):
    inputs = Input(input_size)

    conv1 = conv_layer(64, 3)(inputs)
    conv1 = conv_layer(64, 3)(conv1)
    drop1 = Dropout(dropout_level)(conv1)
    pool1 = MaxPooling3D(pool_size=(2, 2, 2))(drop1)
    conv2 = conv_layer(128, 3)(pool1)
    conv2 = conv_layer(128, 3)(conv2)
    drop2 = Dropout(dropout_level)(conv2)
    pool2 = MaxPooling3D(pool_size=(2, 2, 2))(drop2)
    conv3 = conv_layer(256, 3)(pool2)
    conv3 = conv_layer(256, 3)(conv3)
    drop3 = Dropout(dropout_level)(conv3)
    pool3 = MaxPooling3D(pool_size=(2, 2, 2))(drop3)
    conv4 = conv_layer(512, 3)(pool3)
    conv4 = conv_layer(512, 3)(conv4)
    drop4 = Dropout(dropout_level)(conv4)
    pool4 = MaxPooling3D(pool_size=(2, 2, 2))(drop4)
    conv5 = conv_layer(1024, 3)(pool4)
    conv5 = conv_layer(1024, 3)(conv5)
    drop5 = Dropout(dropout_level)(conv5)

    # Decoder
    up6 = conv_layer(512, 2)(UpSampling3D(size = (2,2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 4)
    drop6 = Dropout(dropout_level)(merge6)
    conv6 = conv_layer(512, 3)(drop6)
    conv6 = conv_layer(512, 3)(conv6)
    
    up7 = conv_layer(256, 2)(UpSampling3D(size = (2,2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 4)
    drop7 = Dropout(dropout_level)(merge7)
    conv7 = conv_layer(256, 3)(drop7)
    conv7 = conv_layer(256, 3)(conv7)

    up8 = conv_layer(128, 2)(UpSampling3D(size = (2,2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 4)
    drop8 = Dropout(dropout_level)(merge8)
    conv8 = conv_layer(128, 3)(drop8)
    conv8 = conv_layer(128, 3)(conv8)

    up9 = conv_layer(64, 2)(UpSampling3D(size = (2,2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 4)
    drop9 = Dropout(dropout_level)(merge9)
    conv9 = conv_layer(64, 3)(drop9)
    conv9 = conv_layer(64, 3)(conv9)
    conv9 = conv_layer(2, 3)(conv9)
    conv10 = Conv3D(1, 1, activation = 'linear')(conv9)

    model = Model(input= inputs, output= conv10)
    model.compile(loss="mean_squared_error", optimizer="adam")
    return model

In [8]:
# Checkpoint
check_path = 'gdrive/My Drive/NU_Rad/models/3d_unet_mse_model'
checkpoint = ModelCheckpoint(check_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

model = unet_model()
model.fit(X_train, y_train, batch_size=4, validation_split=0.1, epochs = 3, use_multiprocessing=True, callbacks=callbacks_list)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:57: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`


Train on 16 samples, validate on 2 samples
Epoch 1/3


ResourceExhaustedError: ignored